Aaron Negrillo<br>
IDSN 544, Fall 2024<br>
anegrill@usc.edu<br>

## Excercise 1: Build your own model

Using the example code from fast.ai lesson 1, "Is it a bird? Creating a model from your own data," build your own visual learner model.Using the example code from fast.ai lesson 1, "Is it a bird? Creating a model from your own data," build your own visual learner model.

In [ ]:
#NB: Kaggle requires phone verification to use the internet or a GPU. If you haven't done that yet, the cell below will fail
#    This code is only here to check that your internet is enabled. It doesn't do anything else.
#    Here's a help thread on getting your phone number verified: https://www.kaggle.com/product-feedback/135367

import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

In [ ]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search

## Model Context: I'm feeling a bit hungry

For cooking enthusiasts and hobby gardeners alike, growing your own herbs can be a rewarding experience. However, everyone occasionally needs a bit of assistance.

A visual model for herb identification can greatly enhance this process by accurately identifying herbs, ensuring that recipes remain authentic and flavorful. It also aids in distinguishing between herbs and similar plants, simplifying garden management. Additionally, visual identification training offers an engaging way to deepen one's knowledge and recognition of herbs.

### Step 1: Loading modules and defining functions

In [ ]:
from duckduckgo_search import DDGS #ddg_images (Deprecated 11-2023)
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    #return L(ddg_images(term, max_results=max_images)).itemgot('image')
    # update module reference
    return L(DDGS().images(term,max_results=max_images)).itemgot('image')

### Step 2: Finding a sample image

In [ ]:
urlsExample = search_images('basil plant', max_images=1)
urlsExample[0]

In [ ]:
from fastdownload import download_url
destExample = 'basil.jpg'
download_url(urlsExample[0], destExample, show_progress=False)

from fastai.vision.all import *
im = Image.open(destExample)
im.to_thumb(256,256)

### Step 3: Building a dataset, cleaning, and some minor validation

In [ ]:
#shutil.rmtree(pathExample) -- just some path debugging (o_o)
searchesExample = 'basil','oregano','thyme','rosemary'
pathExample = Path('basil_or_not')
from time import sleep

for o in searchesExample:
    destExample = (pathExample/o)
    destExample.mkdir(exist_ok=True, parents=True)
    
    download_images(destExample, urls=search_images(f'{o} plant'))
    # here 'urls' is a parameter we are setting a value to
    sleep(10)  # Pause between searches to avoid over-loading server
    
    download_images(destExample, urls=search_images(f'{o} herb'))
    sleep(10)
    
    download_images(destExample, urls=search_images(f'{o} potted'))
    sleep(10)
    
    resize_images(pathExample/o, max_size=400, dest=pathExample/o)

In [ ]:
failedExample = verify_images(get_image_files(pathExample))
failedExample.map(Path.unlink)
len(failedExample)

In [ ]:
# # List all files and directories in destExample
contents = os.listdir(destExample)
print(contents)

len(contents)

### Step 4: Building our DataBlock and training our model

In [ ]:
dlsExample = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(pathExample, bs=32) #call the image path here

dlsExample.show_batch(max_n=6)

In [ ]:
learnExample = vision_learner(dlsExample, resnet18, metrics=error_rate)
learnExample.fine_tune(3)

### Step 5: Evaluating how our model performs

In [ ]:
is_basil,_,probs = learnExample.predict(PILImage.create('basil.jpg'))
print(f"This herb is: {is_basil}.")
print(f"Probability it's basil: {probs[0]:.4f}")